<a href="https://colab.research.google.com/github/Aleyucra74/TCC-CCO-2023/blob/main/simple_download_dataraw_into_s3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pyspark boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 47.1 MB/s eta 0:00:00


In [11]:
import os
import boto3
import zipfile
import urllib.request

In [3]:
#downloading archives
#time executed 5'6''
#!curl -L -o test2.zip https://www.dropbox.com/sh/75v5ehq4cdg5g5g/AAD5w76F_SZLdBgxVdvko-z5a/UCF_Crimes-Train-Test-Split.zip?dl=1
urls = ['https://www.dropbox.com/sh/75v5ehq4cdg5g5g/AABJtkTnNc8LcVTfH1gE_uFoa/Anomaly-Videos-Part-1.zip?dl=1',
       'https://www.dropbox.com/sh/75v5ehq4cdg5g5g/AAAbdSEUox64ZLgVAntr2WgSa/Anomaly-Videos-Part-2.zip?dl=1',
       'https://www.dropbox.com/sh/75v5ehq4cdg5g5g/AAAgpsRNSHI_BtRnSCxxR7j9a/Anomaly-Videos-Part-3.zip?dl=1',
       'https://www.dropbox.com/sh/75v5ehq4cdg5g5g/AABqY-3fJSmSMafFIlJXRE-9a/Anomaly-Videos-Part-4.zip?dl=1']

for index, link in enumerate(urls):
  urllib.request.urlretrieve(link, f"/content/test{index}.zip")

In [4]:
#unziping files
listzip = os.listdir('/content/')

for index, zips in enumerate(listzip):
  if zips[-4:] == ".zip":
    with zipfile.ZipFile(zips, 'r') as zip_ref:
      zip_ref.extractall('/content/')
    os.remove("/content/"+zips)

In [5]:
#removing unnecessary files
# NOT INCLUDE
files_removed = ['Abuse','Arrest','Arson','Explosion','Fighting','RoadAccidents']

listfiles = os.listdir('/content/')

for index, file_name in enumerate(listfiles):
  if file_name[:7] == "Anomaly":
    for name in os.listdir("/content/"+file_name):
      if name in files_removed:
        os.system(f"rm -rf /content/{file_name}/{name}")

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [26]:
#uploading videos into S3 bucket
##secrets keys
aws_access_key_id = 'SECRET_KEY_ID'
aws_secret_access_key = 'SECRET_ACCESS_KEY'
aws_session_token= 'SESSION_TOKEN'
bucket_name = 's3-data-raw-tcc'

s3 = boto3.client('s3', region_name="us-east-1", aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, aws_session_token=aws_session_token)

In [18]:
file_list = []
for root, dirs, files in os.walk('/content/'):
    for file in files:
        if file[-4:] == ".mp4":
          file_path = os.path.join(root, file)
          file_list.append(file_path)

In [27]:
for file_path in file_list:
    # Determine o caminho no S3 onde o arquivo será armazenado
    s3_path = os.path.relpath(file_path, '/content/').replace('\\', '/')
    s3_path = f'{bucket_name}/{s3_path}'

    # Faça o upload do arquivo para o S3
    s3.upload_file(file_path, bucket_name, s3_path)

    print(f'O arquivo {s3_path} foi enviado com sucesso para o bucket {bucket_name}')

O arquivo s3-data-raw-tcc/Anomaly-Videos-Part-3/Shooting/Shooting025_x264.mp4 foi enviado com sucesso para o bucket s3-data-raw-tcc
O arquivo s3-data-raw-tcc/Anomaly-Videos-Part-3/Shooting/Shooting015_x264.mp4 foi enviado com sucesso para o bucket s3-data-raw-tcc
O arquivo s3-data-raw-tcc/Anomaly-Videos-Part-3/Shooting/Shooting036_x264.mp4 foi enviado com sucesso para o bucket s3-data-raw-tcc
O arquivo s3-data-raw-tcc/Anomaly-Videos-Part-3/Shooting/Shooting046_x264.mp4 foi enviado com sucesso para o bucket s3-data-raw-tcc
O arquivo s3-data-raw-tcc/Anomaly-Videos-Part-3/Shooting/Shooting013_x264.mp4 foi enviado com sucesso para o bucket s3-data-raw-tcc
O arquivo s3-data-raw-tcc/Anomaly-Videos-Part-3/Shooting/Shooting031_x264.mp4 foi enviado com sucesso para o bucket s3-data-raw-tcc
O arquivo s3-data-raw-tcc/Anomaly-Videos-Part-3/Shooting/Shooting006_x264.mp4 foi enviado com sucesso para o bucket s3-data-raw-tcc
O arquivo s3-data-raw-tcc/Anomaly-Videos-Part-3/Shooting/Shooting020_x264.mp